In [ ]:
import os
os.chdir("../")

In [ ]:
from datasets import load_dataset
import pickle
from src import CluBartModelForGenerativeQuestionAnswering
from data import WikiTQWithReasonAsInputDataset
import json
from utils import process_config
from tqdm import tqdm
import torch
import torch.nn as nn

In [ ]:
with open("configs/wiki_tq/tapex.json", "rb") as f:
    config = json.load(f)
config = process_config(config)

In [ ]:
model = CluBartModelForGenerativeQuestionAnswering(config)

In [ ]:
model.load_state_dict(torch.load("/dev/shm/tabllm/logs/table_question_answering_omnitab_clustering_with_reason_as_input_bs8/checkpoints/epoch=30.pt", map_location="cpu"))

In [ ]:
dataset = load_dataset("wikitablequestions")
test_dataset = WikiTQWithReasonAsInputDataset(dataset=dataset, config=config, data_type="test")

In [ ]:
tokenizer = test_dataset.tokenizer

In [ ]:

import re
import unicodedata
import numpy as np
import pandas as pd
from tqdm import tqdm
from codecs import open
from math import isnan, isinf
from abc import ABCMeta, abstractmethod

def normalize(x):

    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')

    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)

    while True:
        
        old_x = x

        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        
        if x == old_x:
            break
    
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    
    return x


class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.

        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.

        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):

        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        
        self._year = year
        self._month = month
        self._day = day
        
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        
        assert isinstance(other, Value)
        
        if self.normalized == other.normalized:
            return True
        
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.

        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.

    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """

    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    
    if not corenlp_value:
        corenlp_value = original_string
    
    # Number?
    amount = NumberValue.parse(corenlp_value)
    
    if amount is not None:
        return NumberValue(amount, original_string)
    
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values

    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """

    # Check size
    if len(target_values) != len(predicted_values):
        return False
    
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    
    return True


def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash

    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')


def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)

    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]

In [ ]:
model.to("cuda:0")

In [ ]:
def predict(index):

    input_ids, attention_mask, token_type_ids, decode_input_ids, labels = test_dataset.__getitem__(index)

    actual_answer = tokenizer.decode(labels[labels != -100], skip_special_tokens = True)
    predicted_ids = model.model.generate(input_ids = input_ids.unsqueeze(0).to("cuda:0"), attention_mask = attention_mask.unsqueeze(0).to("cuda:0"), 
                                         num_beams = 3, early_stopping = True, max_length = 64).squeeze().detach().cpu()

    predicted_answer = tokenizer.decode(predicted_ids, skip_special_tokens = True)
    
    gold = [x.strip() for x in actual_answer.split(",")]
    pred = [x.strip() for x in predicted_answer.split(",")]

    pred = to_value_list(pred)
    gold = to_value_list(gold)

    verdict = check_denotation(gold, pred)

    return predicted_answer, actual_answer, verdict

In [ ]:
incorrect_indices = []
predicted_answers = []

In [ ]:
for i in tqdm(range(len(test_dataset)), position = 0, leave = True, total = len(test_dataset)):
    predicted_answer, actual_answer, verdict = predict(i)

    if not verdict:
        incorrect_indices.append(i)
        predicted_answers.append(predicted_answer)


In [ ]:
correct_indices = []
predicted_answers = []

In [ ]:
for i in tqdm(range(len(test_dataset)), position = 0, leave = True, total = len(test_dataset)):
    predicted_answer, actual_answer, verdict = predict(i)

    if verdict:
        correct_indices.append(i)
        predicted_answers.append(predicted_answer)


In [ ]:
with open("datasets/omnitab_best_model_incorrect_indices.pkl", "wb") as f:
    pickle.dump(incorrect_indices, f)

In [ ]:
with open("datasets/omnitab_best_model_incorrect_predicted_answers.pkl", "wb") as f:
    pickle.dump(predicted_answers, f)

In [ ]:
import pickle
import pandas as pd
from datasets import load_dataset
dataset = load_dataset("wikitablequestions")
with open("datasets/omnitab_best_model_incorrect_indices.pkl", "rb") as f:
    incorrect_indices = pickle.load(f)

with open("datasets/omnitab_best_model_incorrect_predicted_answers.pkl", "rb") as f:
    predicted_answers = pickle.load(f)

In [ ]:
from data import WikiTQWithReasonAsInputDataset
import json
from utils import process_config

with open("configs/wiki_tq/tapex.json", "rb") as f:
    config = json.load(f)
config = process_config(config)

test_dataset = WikiTQWithReasonAsInputDataset(dataset=dataset, config=config, data_type = "test")

In [ ]:
for i in range(len(incorrect_indices)):
    idx = incorrect_indices[i]
    if i == 212:
        print(idx)
        question = dataset["test"][idx]["question"]
        answer = test_dataset.text_output[idx]
        predicted_answer = predicted_answers[i]
        reason = test_dataset.reasons[idx]
        table = test_dataset.table[idx]

        print(f"Question: \t\t{question}")
        print(f"Actual Answer: \t\t{answer}")
        print(f"Predicted Answer: \t{predicted_answer}")
        print(f"Reason: \t\t{reason}" )
        print(f"Table:")
        display(table)

        table_column_names = table.columns.tolist()
        table_content_values = table.values.tolist()

        table_dict = {"header": table_column_names, "rows": table_content_values}
        print(table_dict)

In [ ]:
import pickle
with open("datasets/test_wiki_tq_reason.pkl", "rb") as f:
    reasons_with_answer = pickle.load(f)

In [ ]:
reasons_with_answer[2830]

In [ ]:
for i in range(len(correct_indices)):
    idx = correct_indices[i]
    if i == 529:
        print(idx)
        question = dataset["test"][idx]["question"]
        answer = test_dataset.text_output[idx]
        predicted_answer = predicted_answers[i]
        reason = test_dataset.reasons[idx]
        table = test_dataset.table[idx]

        print(f"Question: \t\t{question}")
        print(f"Actual Answer: \t\t{answer}")
        print(f"Predicted Answer: \t{predicted_answer}")
        print(f"Reason: \t\t{reason}" )
        print(f"Table:")
        display(table)

        table_column_names = table.columns.tolist()
        table_content_values = table.values.tolist()

        table_dict = {"header": table_column_names, "rows": table_content_values}
        print(table_dict)

# OmniTab Error cases table truncation analysis

In [ ]:
import pickle
from transformers import AutoTokenizer
from datasets import load_dataset

In [ ]:
import pandas as pd

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/tapex-large-finetuned-wtq")

In [ ]:
with open("datasets/omnitab_best_model_incorrect_indices.pkl", "rb") as f:
    incorrect_indices = pickle.load(f)

In [ ]:
test_dataset = load_dataset("wikitablequestions")["test"]

In [ ]:
input_lengths = []

In [ ]:
from tqdm import tqdm

In [ ]:
for idx in tqdm(incorrect_indices, position = 0, leave = True, total = len(incorrect_indices)):
    question = test_dataset["question"][idx]
    table = test_dataset["table"][idx]
    answers = test_dataset["answers"][idx]
    
    table_column_names = table["header"]
    table_content_values = table["rows"]

    table_df = pd.DataFrame.from_dict({str(col).lower(): [str(table_content_values[j][i]).lower() for j in range(len(table_content_values))] for i, col in enumerate(table_column_names)})

    x = tokenizer.encode(table_df, question)
    input_lengths.append(len(x))


In [ ]:
import numpy as np
np.array(input_lengths)[np.array(input_lengths) > 1024].shape